VERTRAUENSINTERVALL
=====================

In diesem notebook wird aus Messdaten der Mittelwert gebildet, die Standardabweichung berechnet und ein m%-Vertrauensintervall gebildet und ausgegeben.
Der Code ist praktisch identisch mit dem Skript confinterval.py.

Als erstes muss sichergestellt werden, dass die Messdaten in einem .txt file im working directory abgespeichert sind. D.h. in dem Verzeichnis, in dem dieses notebook abgespeichert liegt.

Nun werden die erforderlichen libraries und Funktionen importiert:

In [2]:
#!/bin/python
# Geschrieben 10/2020, Henry Korhonen henryk@ethz.ch, basierend auf Matlabskripten von Martin Willeke. Hinweise, Bemerkungen und Vorschläge bitte an henryk@ethz.ch.

import pandas as pd
import numpy as np
from scipy.stats import t
from math import sqrt

Jetzt müssen die Daten aus dem .txt file ausgelesen und in einen sog. Dataframe (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html für weitere Informationen) geschrieben werden. Dies passiert in der ersten Zeile. Ganz wichtig: der Dateiname (hier "test-mittelwert.txt") muss hardcoded werden, d.h. das erste Argument von read_table() muss direkt im code angepasst werden. Die Messdaten sollten mit Leerzeichen getrennt werden, andernfalls müsste man das Argument sep anpassen.

Aus dem Dataframe werden die Werte der spalte "y" extrahiert. Dies ist ein markanter Unterschied zu Matlab: hier werden "Spalten" nicht mit deren jeweiligen Nummern, sondern mit frei wählbaren Namen (names, string als input) "angesteuert".

Die Anzahl Datenpunkte wird in der Variable N gespeichert und ausgegeben.

In [3]:
xy = pd.read_table('test-mittelwert.txt', names=['x','y'], sep=r'\s+') # Lesen der Daten, erstellen eines Dataframes. Als Separator kommt hier eine unbestimmte Anzahl Leerschläge in Frage. Andernfalls "sep" anpassen.

y = xy['y'] # Relevante Daten aus dem Dataframe extrahieren. Achtung: "names" in pd.read_table gibt der ersten Spalte den Namen x und der zweiten y. Unbedingt sicherstellen, dass die richtigen Daten extrahiert werden!

N = len(y) # Anzahl Datenpunkte ermitteln.
print(f'Anzahl Datenpunkte: {N}')

Anzahl Datenpunkte: 10


Nun wird der Mittelwert generiert und ausgegeben.

In [4]:
mwert = y.mean() # Mittelwert aus dem Dataframe y bestimmen.

print(f'Mittelwert: {mwert}')

Mittelwert: 2685.8


Interessant ist jetzt noch die Standardabweichung (STD, STandard Deviation). Hier muss zwischen der STD einer Einzelmessung und derer des Mittelwertes unterschieden werden. Im code bedeutet das bloß eine weitere Zeile, in der durch die Quadratwurzel der Anzahl Messungen geteilt wird.

Achtung: Default ist in der ersten Zeile ddof=0! (Der Nenner wird so zu N anstatt zum gewünschten N-1)
ddof muss auf 2 gesetzt werden, für Parameter für eine lin. Reg. (ystd wäre dann z.B. die Standardabweichung der Steigung oder des Achsenabschnitts aus einer lin. Reg.)

In [5]:
ystd = np.std(y, ddof=1) # Standardabweichung der EINZELMESSUNG. 

ystd_mean = ystd/sqrt(N) # Standardabweichung des MITTELWERTES

Nun da diese beiden Werte bekannt sind (noch nicht ausgegeben), kann ein m%-Vertrauensintervall berechnet werden.
p (= m/100) muss hardcoded werden. Bei 97.5% wäre bspw. p = 0.975.

In [6]:
p = 0.95
p100 = p*100
intval = t.interval(p,N-1,mwert, ystd_mean)
# t.interval aus der Library scipy.stats macht das, was unten steht bereits elegant in einer Zeile.

tinv = t.ppf((1+p)/2,N-1) # Berechnung des Student-T Faktors
halb = ystd_mean*tinv
# Hier wird die Standardabweichung des MITTELWERTES (s. Messfehlerskript) benutzt!

Schließlich wird das ganze noch ausgegeben. Nota bene: man könnte auch anstelle von intval einfach mwert+halb und mwert-halb wählen, das Ergebnis bliebe dasselbe. intval ist nur da, um zu zeigen, dass viele "wissenschaftliche" libraries wie scipy oder numpy bereits mit sehr nützlichen Funktionen ausgestattet sind.

Et voilà.

In [7]:
print(f'Das {p100}-% Vertrauensintervall ist: {intval}')


print(f'Anders ausgedrückt: {mwert} +- {halb}')

Das 95.0-% Vertrauensintervall ist: (2533.4428166109137, 2838.1571833890866)
Anders ausgedrückt: 2685.8 +- 152.35718338908646
